In [3]:
import pandas as pd
import numpy as np
import uproot3
from sklearn import preprocessing
import tables
from tqdm import tqdm
from timeit import default_timer as timer
#profiler = pprofile.Profile() 


l1 = np.arange(0,200, 100)
l2 = np.arange(100,300, 100)

event_start_stop = list(zip(l1, l2))
print(event_start_stop)

def redefine_layer_id(volume_id, layer_id): 
    """Instead of numbering by volume and then layer, so each volume counts layer from 1, make every layer a unique number"""
    # left side endcap
    if volume_id == 1:
        layer_id = layer_id + 4
    # right side endcap 
    elif volume_id == 3:
        layer_id = layer_id + 4+12
    return layer_id

def create_unique_label_background_tracks(x, y):
    x = -le.transform(y)
    return x
#with profiler: 



[(0, 100), (100, 200)]


In [67]:
%%time
data['layer_id'] = np.where(data['volume_id'] == 1, data['layer_id'] + 4, data['layer_id'])
data['layer_id'] = np.where(data['volume_id'] == 3, data['layer_id'] + 4 + 12, data['layer_id'])

CPU times: user 15.7 ms, sys: 1.05 ms, total: 16.8 ms
Wall time: 10.3 ms


In [66]:
%%time
file = uproot3.open("ntuple_PU200_numEvent1000.root")[b'ntuplizer;1'][b'tree;83']
data = file.pandas.df(["*"], flatten=True,  entrystart=0, entrystop=1)   # not the default


CPU times: user 171 ms, sys: 988 µs, total: 172 ms
Wall time: 172 ms


In [68]:
%%time 
background = data[data['sim_id']==-1]
tracks = data[data['sim_id']!=-1]
del tracks['sim_id']
print(3, timer())
#profiler.print_stats()
indices = tracks.groupby(['particle_id', 'sim_pt']).agg({'x':'count'}).reset_index()
del indices['x']
indices = indices.reset_index().rename(columns = {'index':'sim_id'})
#indices['sim_id'] = indices['sim_id'] + 1
tracks.merge(indices, on = ['particle_id', 'sim_pt'], how = 'right')


3 604746.100107299
CPU times: user 41.5 ms, sys: 1.01 ms, total: 42.5 ms
Wall time: 39 ms


,run,lumi,evt,nhit,hit_n,hit_id,x,y,z,particle_id,...,gen_phi,sim_type,sim_dxy_sig,sim_pt,sim_eta,sim_phi,volume_id,layer_id,module_id,sim_id
0,1,89,8805,136545,136545,39082,9.813801,5.543828,-52.511002,-1,...,0.000000,0,2.305151,0.113190,-2.129276,0.544111,1,8,2,0
1,1,89,8805,136545,136545,39120,9.904799,5.569283,-52.910999,-1,...,0.000000,0,0.988931,0.113190,-2.129276,0.544111,1,8,3,0
2,1,89,8805,136545,136545,44738,13.164094,5.945054,-66.708000,-1,...,0.000000,0,5.174963,0.113190,-2.129276,0.544111,1,9,3,0
3,1,89,8805,136545,136545,62207,9.487366,-4.402370,-175.600006,-1,...,0.000000,0,1.636529,0.113190,-2.129276,0.544111,1,13,26,0
4,1,89,8805,136545,136545,62273,9.597630,-4.546273,-174.800003,-1,...,0.000000,0,1.352965,0.113190,-2.129276,0.544111,1,13,27,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,1,89,8805,136545,136545,80529,-5.800885,-0.694762,25.600000,909,...,0.441827,0,0.360144,16.823303,0.384985,0.441827,3,17,11,209
1440,1,89,8805,136545,136545,85992,-8.412100,-0.122645,31.375999,909,...,0.441827,0,0.187300,16.823303,0.384985,0.441827,3,18,17,209
1441,1,89,8805,136545,136545,87009,-8.783377,-0.012305,32.175999,909,...,0.441827,0,0.642712,16.823303,0.384985,0.441827,3,18,13,209
1442,1,89,8805,136545,136545,86971,-8.966237,0.046981,32.576000,909,...,0.441827,0,0.266158,16.823303,0.384985,0.441827,3,18,12,209


In [69]:
%%time 
le = preprocessing.LabelEncoder()
le.fit(background['sim_pt'].unique())
background['sim_id'] = create_unique_label_background_tracks(background['sim_id'], background['sim_pt'])
final_df = tracks.append(background)


CPU times: user 93.8 ms, sys: 2.85 ms, total: 96.6 ms
Wall time: 94.8 ms


In [78]:
%%time 
final_df.to_hdf( "ntuple_PU200/new_simids/ntuple_PU200_event{}.h5".format(1), complevel=9, key='df', mode='w')


CPU times: user 31.5 ms, sys: 16.9 ms, total: 48.4 ms
Wall time: 47.9 ms


In [82]:
%%time
from tqdm import tqdm
import pprofile 
pd.options.mode.chained_assignment = None  # default='warn'

file = uproot3.open("ntuple_PU200_numEvent1000.root")[b'ntuplizer;1'][b'tree;83']

profiler = pprofile.Profile()

for i in tqdm(range(1,10)): 
    data = file.pandas.df(["*"], flatten=True,  entrystart=i, entrystop=i+1)   # not the default
    #print(1, timer())
  # profiler.print_stats()
    #data = file.pandas.df(["*"], flatten=True, entrystart=0, entrystop=100)
    data['r'] = np.sqrt(data['x']**2 + data['y']**2)
    #data['new_layer_ids'] = data.apply(lambda x: redefine_layer_id(x.volume_id, x.layer_id), axis=1)
    data['new_layer_id'] = np.where(data['volume_id'] == 1, data['layer_id'] + 4, data['layer_id'])
    data['new_layer_id'] = np.where(data['volume_id'] == 3, data['layer_id'] + 4 + 12, data['layer_id'])
    #print(2, timer())
    #profiler.print_stats()

    background = data[data['sim_id']==-1]
    tracks = data[data['sim_id']!=-1]
    del tracks['sim_id']
    #print(3, timer())
    #profiler.print_stats()
    indices = tracks.groupby(['particle_id', 'sim_pt']).agg({'x':'count'}).reset_index()
    #print(indices)
    del indices['x']
    indices = indices.reset_index().rename(columns = {'index':'sim_id'})
    #print(indices)
    #indices['sim_id'] = indices['sim_id'] + 1
    tracks = tracks.merge(indices, on = ['particle_id', 'sim_pt'], how = 'right')
    #print(tracks['sim_id'])
    #profiler.print_stats()
    le = preprocessing.LabelEncoder()
    le.fit(background['sim_pt'].unique())
    background['sim_id'] = create_unique_label_background_tracks(background['sim_id'], background['sim_pt'])
    final_df = tracks.append(background)
    final_df = final_df.reset_index(drop=True)
    final_df.to_hdf( "ntuple_PU200/new_simids/ntuple_PU200_event{}.h5".format(i), complevel=9, key='df', mode='w')
    #profiler.print_stats() 


100%|██████████| 9/9 [00:24<00:00,  2.68s/it]

CPU times: user 24 s, sys: 101 ms, total: 24.1 s
Wall time: 24.2 s


In [79]:
d = pd.read_hdf('ntuple_PU200/new_simids/ntuple_PU200_event4.h5')

In [83]:
d

,run,lumi,evt,nhit,hit_n,hit_id,x,y,z,particle_id,...,sim_type,sim_dxy_sig,sim_pt,sim_eta,sim_phi,volume_id,layer_id,module_id,r,sim_id
0,1,89,8802,138944,138944,113064,-0.099937,15.487688,109.257004,-1,...,0,2.005562,0.100581,1.596073,1.252535,3,23,8,15.488010,0
1,1,89,8802,138944,138944,113099,0.016103,15.368772,108.857002,-1,...,0,12.931124,0.100581,1.596073,1.252535,3,23,9,15.368780,0
2,1,89,8802,138944,138944,120481,-5.847015,9.827031,174.800003,-1,...,0,2.242001,0.100581,1.596073,1.252535,3,25,10,11.434952,0
3,1,89,8802,138944,138944,125138,-1.371006,12.246107,200.759003,-1,...,0,1.322131,0.100581,1.596073,1.252535,3,26,8,12.322613,0
4,1,89,8802,138944,138944,131239,-5.353776,16.821964,230.968994,-1,...,0,0.527450,0.100581,1.596073,1.252535,3,27,11,17.653368,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138939,1,89,8802,138944,138944,138939,21.713118,-1.294127,265.200012,-1,...,1,0.000000,0.386141,2.284478,0.292889,3,28,48,21.751650,-9594
138940,1,89,8802,138944,138944,138940,24.274706,-1.468288,265.200012,-1,...,1,0.000000,0.345563,2.593338,-1.565621,3,28,48,24.319071,-8462
138941,1,89,8802,138944,138944,138941,21.418783,-1.415135,265.200012,-1,...,1,0.000000,0.469033,3.204305,0.204243,3,28,48,21.465481,-11504
138942,1,89,8802,138944,138944,138942,24.343948,-1.942580,265.200012,-1,...,1,0.000000,1.153086,2.501649,0.006369,3,28,48,24.421331,-16946


In [52]:
len(d['sim_id'].unique())


18414

In [81]:
d['new_layer_id']

KeyError: 'new_layer_id'

In [62]:
d

,run,lumi,evt,nhit,hit_n,hit_id,x,y,z,particle_id,...,sim_type,sim_dxy_sig,sim_pt,sim_eta,sim_phi,volume_id,layer_id,module_id,r,sim_id
0,1,89,8802,138944,138944,113064,-0.099937,15.487688,109.257004,-1,...,0,2.005562,0.100581,1.596073,1.252535,3,23,8,15.488010,0
1,1,89,8802,138944,138944,113099,0.016103,15.368772,108.857002,-1,...,0,12.931124,0.100581,1.596073,1.252535,3,23,9,15.368780,0
2,1,89,8802,138944,138944,120481,-5.847015,9.827031,174.800003,-1,...,0,2.242001,0.100581,1.596073,1.252535,3,25,10,11.434952,0
3,1,89,8802,138944,138944,125138,-1.371006,12.246107,200.759003,-1,...,0,1.322131,0.100581,1.596073,1.252535,3,26,8,12.322613,0
4,1,89,8802,138944,138944,131239,-5.353776,16.821964,230.968994,-1,...,0,0.527450,0.100581,1.596073,1.252535,3,27,11,17.653368,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(4, 138939)",1,89,8802,138944,138944,138939,21.713118,-1.294127,265.200012,-1,...,1,0.000000,0.386141,2.284478,0.292889,3,28,48,21.751650,-9594
"(4, 138940)",1,89,8802,138944,138944,138940,24.274706,-1.468288,265.200012,-1,...,1,0.000000,0.345563,2.593338,-1.565621,3,28,48,24.319071,-8462
"(4, 138941)",1,89,8802,138944,138944,138941,21.418783,-1.415135,265.200012,-1,...,1,0.000000,0.469033,3.204305,0.204243,3,28,48,21.465481,-11504
"(4, 138942)",1,89,8802,138944,138944,138942,24.343948,-1.942580,265.200012,-1,...,1,0.000000,1.153086,2.501649,0.006369,3,28,48,24.421331,-16946
